In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import os


In [ ]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Print working directory
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize((160, 160*8)),  # Resize to 80x(80*8)
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load datasets
data_dir = "datafolder"
train_dir = os.path.join(data_dir, "frames_no_bg_dropout_train")
test_dir = os.path.join(data_dir, "frames_no_bg_dropout_test")

# Print dataset directories
print(f"Train dataset directory: {train_dir}")
print(f"Test dataset directory: {test_dir}")

# Check if directories exist
if not os.path.exists(train_dir):
    print(f"Train directory does not exist: {train_dir}")
if not os.path.exists(test_dir):
    print(f"Test directory does not exist: {test_dir}")

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# Show how many positive samples and negative samples are in the datasets
def count_samples(loader):
    pos = 0
    neg = 0
    for data, target in loader:
        pos += target.sum()
        neg += (1 - target).sum()
    return pos, neg


train_pos, train_neg = count_samples(train_loader)
test_pos, test_neg = count_samples(test_loader)

print(f"Train dataset: {train_pos} positive samples, {train_neg} negative samples")
print(f"Test dataset: {test_pos} positive samples, {test_neg} negative samples")

In [ ]:
import matplotlib.pyplot as plt
import random

def show_sample_images(dataset, num_samples=10):
    indices = random.sample(range(len(dataset)), num_samples)
    fig, axes = plt.subplots(num_samples, 1, figsize=(5, 15))
    for i, idx in enumerate(indices):
        image, label = dataset[idx]
        image = image.permute(1, 2, 0)  # Move channel dimension to the end
        image = image * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Denormalize
        image = image.numpy()
        axes[i].imshow(image)
        axes[i].set_title(f"Label: {label}")
        axes[i].axis('off')
    plt.show()

# Show sample images from the training dataset
show_sample_images(train_dataset)

In [6]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, writer=None):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update metrics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

        if writer:
            writer.add_scalar("Loss/train", epoch_loss, epoch)
            writer.add_scalar("Accuracy/train", epoch_acc, epoch)
            writer.flush()

    if writer:
        writer.close()

In [ ]:

# Load pretrained ResNet
model = models.resnet18(pretrained=True)  # Use ResNet18; you can also choose ResNet50
num_features = model.fc.in_features  # Number of features in the final layer
model.fc = nn.Linear(num_features, 2)  # Modify the final layer to output 2 classes (good/bad)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=1e-5)

writer = SummaryWriter("runs/golf_swing_classification")

train_model(model, train_loader, criterion, optimizer, num_epochs=10, writer=writer)

In [ ]:
# Save the model
torch.save(model.state_dict(), "golf_swing_classification.pth")

# Test the model
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {correct / total:.4f}")

test_model(model, test_loader)

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, GradCAMElementWise, GradCAMPlusPlus, AblationCAM, ScoreCAM, LayerCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import cv2
import numpy as np

model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(device)
model.eval()

state_dict = torch.load("golf_swing_classification.pth")
model.load_state_dict(state_dict)

# Generate Grad-CAM visualizations
target_layer = [model.layer4[-1]]
# target_layer = [model.layer4[1].conv2]


def generate_gradcam_visualizations(model, dataloader, target_layers, device):
    cam = GradCAMElementWise(model=model, target_layers=target_layers)
    img_count = 0
    for images, labels in dataloader:
        images = images.to(device)
        outputs = model(images)
        predicted_classes = torch.argmax(outputs, dim=1)
        
        for i in range(len(images)):
            targets = [ClassifierOutputTarget(predicted_classes[i].item())]
            grayscale_cam = cam(input_tensor=images[i].unsqueeze(0), targets=targets)

            img = images[i].cpu().numpy().transpose(1, 2, 0)
            img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            img = np.clip(img, 0, 1)
            
            visualization = show_cam_on_image(img, grayscale_cam[0], use_rgb=True)
            cv2.imwrite(f'resnet_gradcam_elementwise_vis_dropout/gradcam_visualization_{img_count}.jpg', visualization)
            img_count += 1

generate_gradcam_visualizations(model, test_loader, target_layer, device)